####  import related modules

In [22]:
import warnings
warnings.filterwarnings("ignore")
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import math
import random

#### File finder

In [23]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add




#### Folder creator

In [24]:
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

folder("temp")

#### Delete unnecessary files

In [25]:
def killthemall():
    them=find_the_way("./temp/",".csv")
    for t in them:
        try:
            os.remove(t)
        except:
            print(f"error about delete {t} file")

In [26]:
# find pcap file in path folder
path="pcaps"
pcaps=find_the_way(path,".pcap")
pcaps

['pcaps\\16-09-29.pcap',
 'pcaps\\16-10-03.pcap',
 'pcaps\\16-11-18.pcap',
 'pcaps\\16-11-22.pcap',
 'pcaps\\18-06-11.pcap',
 'pcaps\\18-06-14.pcap',
 'pcaps\\18-10-13.pcap',
 'pcaps\\18-10-16.pcap']

#### If your computer has difficulty processing large pcap files, you can split them using the following script

In [27]:
#for i in pcaps:
    #if os.path.getsize(i)//(1024*1024) >500:
        #temp=i.replace("\\","/")
        #command=f"editcap  -c 1000000 {temp} {temp.replace("pcaps/","pcaps/smaller_")}"
        #os.system(command)
        #print(f"{i} made smaller")
        #os.remove(temp)

In [28]:
### Renaming tools

In [29]:
# Feature list
features=[['dns.time',  'http.time',  'icmp.data_time_relative',  'icmp.resptime',  'stun.time',  'tcp.ack',  'tcp.ack_raw',  'tcp.analysis.ack_rtt',  'tcp.analysis.initial_rtt',  'tcp.analysis.rto',  'tcp.nxtseq',  'tcp.options.timestamp.tsecr',  'tcp.options.timestamp.tsval',  'tcp.seq',  'tcp.seq_raw',  'tcp.time_delta',  'tcp.time_relative',  'udp.time_delta',  'udp.time_relative',  'arp.hw.size',  'arp.hw.type',  'arp.isannouncement',  'arp.isgratuitous',  'arp.isprobe',  'arp.opcode',  'arp.proto.size',  'arp.seconds-since-duplicate-address-frame',  'dhcp.flags.bc',  'dhcp.hops',  'dhcp.hw.addr_padding',  'dhcp.hw.len',  'dhcp.option.dhcp',  'dhcp.option.dhcp_max_message_size',  'dhcp.option.end',  'dhcp.option.interface_mtu',  'dhcp.option.ip_address_lease_time',  'dhcp.option.padding',  'dhcp.option.rebinding_time_value',  'dhcp.option.renewal_time_value',  'dhcp.secs',  'dhcp.type',  'dns.count.add_rr',  'dns.count.answers',  'dns.count.auth_rr',  'dns.count.queries',  'dns.extraneous',  'dns.extraneous.data',  'dns.extraneous.length',  'dns.flags.authenticated',  'dns.flags.authoritative',  'dns.flags.checkdisable'], ['dns.flags.opcode',  'dns.flags.rcode',  'dns.flags.recavail',  'dns.flags.recdesired',  'dns.flags.response',  'dns.flags.truncated',  'dns.flags.z',  'dns.response_to',  'dns.retransmission',  'dns.retransmit_request',  'dns.retransmit_request_in',  'dns.retransmit_response',  'dns.retransmit_response_in',  'dns.soa.expire_limit',  'dns.soa.minimum_ttl',  'dns.soa.refresh_interval',  'dns.soa.retry_interval',  'dns.soa.serial_number',  'dns.unsolicited',  'eapol.keydes.key_iv',  'eapol.keydes.key_len',  'eapol.keydes.replay_counter',  'eapol.keydes.type',  'eapol.len',  'eapol.type',  'eapol.version',  'eth.dst.ig',  'eth.dst.lg',  'eth.dst.oui',  'eth.fcs.status',  'eth.len',  'eth.padding',  'eth.padding_bad',  'eth.src.ig',  'eth.src.lg',  'eth.src.oui',  'http.chat',  'http.content_length',  'http.content_length_header',  'http.notification',  'http.prev_request_in',  'http.prev_response_in',  'http.request',  'http.request_in',  'http.request_number',  'http.response',  'http.response.code',  'http.response_number',  'icmp.checksum.status',  'icmp.code',  'icmp.ident'], ['icmp.ident_le',  'icmp.resp_to',  'icmp.seq',  'icmp.seq_le',  'icmp.type',  'icmp.unused',  'icmpv6.checksum.status',  'icmpv6.code',  'icmpv6.mldr.nb_mcast_records',  'icmpv6.opt',  'icmpv6.opt.length',  'icmpv6.opt.type',  'icmpv6.reserved',  'icmpv6.type',  'igmp.checksum.status',  'igmp.max_resp',  'igmp.num_grp_recs',  'igmp.version',  'ip.bogus_ip_length',  'ip.len',  'ip.opt.len',  'ip.opt.ra',  'ip.opt.type',  'ip.opt.type.class',  'ip.opt.type.copy',  'ip.opt.type.number',  'ipv6.hlim',  'ipv6.hopopts.nxt',  'ipv6.nxt',  'ipv6.opt.router_alert',  'ipv6.plen',  'ipv6.tclass.dscp',  'ipv6.tclass.ecn',  'ipv6.version',  'ntp.flags.li',  'ntp.flags.mode',  'ntp.flags.vn',  'ntp.ppoll',  'ntp.precision',  'ntp.rootdelay',  'ntp.rootdispersion',  'ntp.stratum',  'stun.att.crc32.status',  'stun.att.error',  'stun.att.error.class',  'stun.att.lifetime',  'stun.attributes',  'stun.length',  'stun.network_version',  'stun.response-to',  'tcp.analysis'], ['tcp.analysis.ack_lost_segment',  'tcp.analysis.acks_frame',  'tcp.analysis.bytes_in_flight',  'tcp.analysis.duplicate_ack_frame',  'tcp.analysis.duplicate_ack_num',  'tcp.analysis.flags',  'tcp.analysis.keep_alive',  'tcp.analysis.keep_alive_ack',  'tcp.analysis.lost_segment',  'tcp.analysis.out_of_order',  'tcp.analysis.push_bytes_sent',  'tcp.analysis.retransmission',  'tcp.analysis.reused_ports',  'tcp.analysis.rto_frame',  'tcp.analysis.spurious_retransmission',  'tcp.analysis.window_update',  'tcp.checksum.status',  'tcp.completeness',  'tcp.connection.fin',  'tcp.connection.fin_active',  'tcp.connection.fin_passive',  'tcp.connection.rst',  'tcp.connection.syn',  'tcp.connection.synack',  'tcp.dstport',  'tcp.fin_retransmission',  'tcp.flags.ack',  'tcp.flags.ae',  'tcp.flags.cwr',  'tcp.flags.ece',  'tcp.flags.fin',  'tcp.flags.push',  'tcp.flags.res',  'tcp.flags.reset',  'tcp.flags.syn',  'tcp.flags.urg',  'tcp.hdr_len',  'tcp.len',  'tcp.options.mss_val',  'tcp.options.sack.count',  'tcp.options.sack.dsack',  'tcp.options.sack.dsack_le',  'tcp.options.sack.dsack_re',  'tcp.options.sack_le',  'tcp.options.sack_perm',  'tcp.options.sack_re',  'tcp.options.wscale.multiplier',  'tcp.options.wscale.shift',  'tcp.pdu.size',  'tcp.reassembled.length',  'tcp.segment.count'], ['tcp.segment.overlap',  'tcp.segments',  'tcp.srcport',  'tcp.stream',  'tcp.urgent_pointer',  'tcp.window_size',  'tcp.window_size_scalefactor',  'tcp.window_size_value',  'tls.alert_message',  'tls.change_cipher_spec',  'tls.handshake.certificates',  'tls.handshake.certificates_length',  'tls.handshake.challenge',  'tls.handshake.challenge_length',  'tls.handshake.cipher_spec_len',  'tls.handshake.cipher_suites_length',  'tls.handshake.ciphersuites',  'tls.handshake.client_point_len',  'tls.handshake.comp_method',  'tls.handshake.comp_methods',  'tls.handshake.comp_methods_length',  'tls.handshake.epms_len',  'tls.handshake.extension.heartbeat.mode',  'tls.handshake.extensions_ec_point_formats',  'tls.handshake.extensions_ec_point_formats_length',  'tls.handshake.extensions_length',  'tls.handshake.extensions_reneg_info_len',  'tls.handshake.extensions_server_name_len',  'tls.handshake.extensions_server_name_list_len',  'tls.handshake.extensions_server_name_type',  'tls.handshake.extensions_supported_groups',  'tls.handshake.extensions_supported_groups_length',  'tls.handshake.g_len',  'tls.handshake.p_len',  'tls.handshake.server_point_len',  'tls.handshake.session_id_length',  'tls.handshake.session_ticket_length',  'tls.handshake.session_ticket_lifetime_hint',  'tls.handshake.sig_hash_alg_len',  'tls.handshake.sig_hash_algs',  'tls.handshake.sig_len',  'tls.handshake.yc_len',  'tls.handshake.ys_len',  'tls.resumed',  'tls.sct.scts_length',  'udp.checksum.status',  'udp.dstport',  'udp.length',  'udp.length.bad',  'udp.possible_traceroute',  'udp.srcport'], ['udp.stream',  'eth.dst',  'eth.src', 'udp.payload',  'stun.cookie',  'stun.id',  'stun.value',  'tcp.options',  'tcp.options.mss','tcp.options.wscale',  'ntp.refid','arp.proto.type',  'dhcp.flags',  'dhcp.flags.reserved',  'dhcp.id',  'dns.flags',  'dns.id',  'eth.fcs',  'eth.type',  'icmp.checksum',  'icmpv6.checksum',  'igmp.checksum',  'igmp.type',  'ip.id',  'ipv6.flow',  'ipv6.tclass',  'ntp.flags',  'stun.att.crc32',  'stun.att.transp',  'stun.type',  'stun.type.class',  'stun.type.method',  'stun.type.method-assignment',  'tcp.checksum',  'tcp.flags',  'tls.handshake.server_curve_type',  'tls.handshake.server_named_curve',  'tls.handshake.version',  'udp.checksum',  'http.connection',  'http.request.method','http.response.code.desc',  'http.response.phrase'      ], ['tcp.flags.str',  'tcp.payload',  '_ws.col.Protocol',  'tls.handshake.sig_hash_alg',  'dhcp.hw.type',  'ip.dsfield',  'ip.flags',  'ipv6.opt.type',  'ipv6.opt.type.rest',  'stun.att.family',  'stun.att.ipv4-xord',  'stun.att.port-xord',  'tcp.segment',  'arp.duplicate-address-detected',  'arp.duplicate-address-frame',  'dns.srv.port',  'dns.srv.priority',  'dns.srv.weight',  'eth.addr.oui',  'eth.ig',  'eth.lg',  'ip.checksum.status',  'ip.dsfield.dscp',  'ip.dsfield.ecn',  'ip.flags.df',  'ip.flags.mf',  'ip.flags.rb',  'ip.frag_offset',  'ip.hdr_len',  'ip.proto',  'ip.ttl',  'ip.version',  'ipv6.opt',  'ipv6.opt.length',  'ipv6.opt.type.action',  'ipv6.opt.type.change',  'stun.att.padding',  'stun.att.port',  'stun.att.reserved',  'tcp.analysis.duplicate_ack',  'tls.record.length']]

# dictionary to convert string to category
strs={"http_connection": {'Close': 1, 'Keep-Alive': 2, 'close': 3, 'keep-alive': 4},
"http_request_method": {'DELETE': 1, 'GET': 2, 'M-SEARCH': 3, 'NOTIFY': 4, 'POST': 5, 'PUT': 6,'SUBSCRIBE':7, 'UNSUBSCRIBE':8},
"http_request_uri_path": {'/agent/upgrade': 1, '/dst': 2, '/queue/getmessage': 3,
'/router/firmware/query.aspx': 4, '/scale/register': 5, '/scale/validate': 6,"/index.php":7,},
"http_response_code_desc": {'Continue': 1, 'Not Found': 2, 'OK': 3, 'Unauthorized': 4,"Precondition Failed":5, "No Content":6,"Forbidden":7,"Found":8,"Moved Temporarily":9 },
 "http_response_phrase": {'Continue': 1, 'Not Found': 2, 'OK': 3, 'Unauthorized': 4,"Precondition Failed":5, "No Content":6,"Forbidden":7,"Found":8,"Moved Temporarily":9},
"dns_srv_proto": {'_dhnap': 1, '_dhnap,_http': 2, '_hap': 3, '_http': 4, '_http,_dhnap': 5,"_dcp":7,"_dcp,_http":8, "_http,_dcp": 9,  '_http,_http': 6},
"tcp_flags_str": {'·······AP··F': 1, '·······AP···': 2, '·······A·R··': 3, '·······A··S·': 4,
    '·······A···F': 5, '·······A····': 6, '·········R··': 7,  '··········S·': 8,'····CE····S·':9, "····C··AP···":10,"···········F":11},
"_ws_col_Protocol": {'EAPOL':1,'DHCP':2,'DNS':3,'TCP':4,'HTTP':5,'ICMP':6,'MDNS':7,'IGMPv3':8,'SSDP':9,'NTP':10,'HTTP/XML':11,'UDP':12,'SSLv2':13,
                     'TLSv1':14,'TLSv1.2':14,'TLSv1.1':14,"TLSv1.3":14,
'ADwin Config':15,'ICMPv6':17,'HTTP/JSON':18,'XID':19,'TFTP':20,'NXP 802.15.4 SNIFFER':21,'IGMPv2':22,'A21':23,'STUN':24,'Gearman':25,
'?KNXnet/IP':26,"? KNXnet/IP":26,'UDPENCAP':27,'ESP':28,'SSL':29,'NBNS':30,'SIP':31,'BROWSER':32,'SABP':33,'ISAKMP':34,'CLASSIC-STUN':35,'Omni-Path':36,'XMPP/XML':37,
'ULP':38,'TFPoverTCP':39,'AX4000':40,'MIH':41,'DHCPv6':42,'TDLS':43,'RTMP':44,'TCPCL':45,'IPA':46,'GQUIC':47,'0x86dd':48,'DB-LSP-DISC':49,'SSLv3':50,
'LLMNR':51,'FB_ZERO':52,'OCSP':53,'IPv4':54,'STP':55,'SSH':56,'KINK':58,'MANOLITO':59,'PKTC':60,'TELNET':61,'RTSP':62,'HCrt':63,'MPTCP':64,
'S101':65,'IRC':66,'AJP13':67,'PMPROXY':68,'PNIO':69,'AMS':70,'ECATF':71,'LLC':72,'TZSP':73,'RSIP':74,'SSHv2':75,'DIAMETER':76,'BFDControl':77,'ASAP':78
,'DISTCC':79,'DISTCC ':79,'LISP':80,'WOW':81,'DTLSv1.0':82,'SNMP':83,'SMB2':84,'SMB':85,'NBSS':86,'UDT':87,'HiQnet':88,'POWERLINK/UDP':89
,'RTP':90,'WebSocket':91,'NAT-PMP':92,'RTCP':93,'Syslog':94,'Portmap':95,'OpenVPN':96,'BJNP':97,'RIPv1':98,'MAC-Telnet':99,'ECHO':100,'ASF':101
,'DAYTIME':102,'SRVLOC':103,'KRB4':104,'CAPWAP-Control':105,'XDMCP':106,'Chargen':107,'RADIUS':108,'L2TP':109,'DCERPC':110,'KPASSWD':111,'H264':112
,'FTP':113,'FTP-DATA':114,'ENIP':115,'RIPv2':116,'ICP':117,"BACnet-APDU":118,"IAX2":119,"RX":120,"HTTP2":121,"SIP/SDP":122,"TIME":123,
"Elasticsearch":124,"RSL":125,"TPCP":126,"IPv6":127,"ARP":128,"TCP/XML":129, "TPLINK-SMARTHOME/JSON" :130, "DB-LSP-DISC/JSON":131, "BFD Control":132, "DISTCC":133,
"TFP over TCP":134,"PCEP":135,"NDPS":136,
"GTP":137,
"TC-NV":138,
"RDPUDP":139,
"OpcUa":140,
"RSVP":16,
"NVMe/TCP":57,
"BFD Echo":141,
"BAT_BATMAN":142,
"GRE":143,
"VxLAN":144,
"RRoCE":145,
"HICP":146,
"MQTT":147,
"VITA 49":148,
"BAT_GW":149,
"0x0006":150,
"0x0009":150,
"0x0000":150,
"GSMTAP":151,
"CBSP":152,
"DNPv72":153,
"DNPv0 ":154,
"DNPv27":155,
"IPP":156,
"PCP v2":157,
"IMAP":158}}
# string to category feature list
str_cols=["http.connection",
"http.request.method",
#"http.request.uri.path",
"http.response.code.desc",
"http.response.phrase",
#"dns.srv.proto",
"tcp.flags.str",
"_ws.col.Protocol"]







# MAC address of devices and their names. It will use for labelling
macs={'d0:52:a8:00:67:5e': 'Smart Things',
 '44:65:0d:56:cc:d3': 'Amazon Echo',
 '70:ee:50:18:34:43': 'Netatmo Welcome',
 'f4:f2:6d:93:51:f1': 'TP-Link Day Night Cloud camera',
 '00:16:6c:ab:6b:88': 'Samsung SmartCam',
 '30:8c:fb:2f:e4:b2': 'Dropcam',
 'ec:1a:59:79:f4:89': 'Belkin Wemo switch',
 '50:c7:bf:00:56:39': 'TP-Link Smart plug',
 '74:c6:3b:29:d7:1d': 'iHome',
 'ec:1a:59:83:28:11': 'Belkin wemo motion sensor',
 '18:b4:30:25:be:e4': 'NEST Protect smoke alarm',
 '70:ee:50:03:b8:ac': 'Netatmo weather station',
 'd0:73:d5:01:83:08': 'Light Bulbs LiFX Smart Bulb',
 '18:b7:9e:02:20:44': 'Triby Speaker',
 'e0:76:d0:33:bb:85': 'PIX-STAR Photo-frame',
 '70:5a:0f:e4:9b:c0': 'HP Printer',
 '08:21:ef:3b:fc:e3': 'Samsung Galaxy Tab',
 'ac:bc:32:d4:6f:2f': 'MacBook',
 '14:cc:20:51:33:ea': 'TPLink Router Bridge LAN',
 '00:17:88:2b:9a:25': 'Phillip Hue Lightbulb',
 #'7c:70:bc:5d:5e:dc': 'Canary Camera',
 '70:88:6b:10:0f:c6': 'Awair air quality monitor'}



In [30]:
#entropy calculation (shannon entropy )
def shannon(hex_string):
    LOG_BASE = 2
   # We determine the frequency of each byte
   # in the dataset and if this frequency is not null we use it for the
   # entropy calculation.
    try:
        binary_data = bytes.fromhex(hex_string)
        dataSize = len(binary_data)
        ent = 0.0
        freq={} 
        for c in binary_data:
            if c in freq:
                freq[c] += 1
            else:
                freq[c] = 1
       # to determine if each possible value of a byte is in the list
        for key in freq.keys():
            f = float(freq[key])/dataSize
            if f > 0: # to avoid an error for log(0)
                ent = ent + f * math.log(f, LOG_BASE)
    except:
        return hex_string
    return -ent

In [31]:
# port number to port classes
def port_class(port):
    port_list=[0,53,67,68,80,123,443,1900,5353,49153]# private port list (0-Reserved,53-DNS, 67-BOOTP server, 68-BOOTP client...)
    if port in port_list: #Is the port number in the list?
        return port_list.index(port)+1 # return the port's index number in the list (actually with index+1)
    elif 0 <= port <= 1023: # return 11 if the port number is in the range 0 :1023
        return 11
    elif  1024 <= port <= 49151 : # return 12 if the port number is in the range 1024:49151
        return 12
    elif 49152 <=port <= 65535 :# return 13 if the port number is in the range 49152:65535
        return 13
    else:# return 0 if no previous conditions are met
        return 0
    

In [32]:
# in some cases there are two values in the network packets, for example two IP layers inside each other. In these cases only 
# the first value is taken and the second value is discarded. Here is a list of features that suffer from this
dubs=["tls.handshake.sig_hash_alg",
"dhcp.hw.type",
"ip.dsfield",
"ip.flags",
"ipv6.opt.type",
"ipv6.opt.type.rest",
"stun.att.family",
"stun.att.ipv4-xord",
"stun.att.port-xord",
"tcp.segment",
"arp.duplicate-address-detected",
"arp.duplicate-address-frame",
"dns.srv.port",
"dns.srv.priority",
"dns.srv.weight",
"eth.addr.oui",
"eth.ig",
"eth.lg",
"ip.checksum.status",
"ip.dsfield.dscp",
"ip.dsfield.ecn",
"ip.flags.df",
"ip.flags.mf",
"tcp.options.sack_le",
"tcp.options.sack_re",
"icmp.unused",
'arp.seconds-since-duplicate-address-frame',
'ip.len',
'ip.id',
"ip.flags.rb",
"ip.frag_offset",
"ip.hdr_len",
"ip.proto",
"ip.ttl",
"ip.version",
"ipv6.opt",
"ipv6.opt.length",
"ipv6.opt.type.action",
"ipv6.opt.type.change",
"stun.att.padding",
"stun.att.port",
"stun.att.reserved",
"tcp.analysis.duplicate_ack",
"tls.record.length"]
#"ip.dst",
#"ip.src",
#"ip.src_host",
#"stun.att.ipv4",
#"dhcp.hw.mac_addr"]


# hexadecimal to decimal function
def comma_cleaner(val):
    clean=[]
    for v in val:
        if "," in str(v):
            temp=v.split(",")
            temp=temp[0]
        else:
            temp=v
        clean.append(temp)  
    return clean


In [33]:
# features with hexadecimal values
hexvalues=["stun.cookie",
"stun.id",
"stun.value",
"tcp.options",
"tcp.options.mss",
#"tcp.options.sack",
"tcp.options.wscale",
"ntp.refid",
#"stun.att.username",
"arp.proto.type",
"dhcp.flags",
"dhcp.flags.reserved",
"dhcp.id",
"dns.flags",
"dns.id",
"eth.fcs",
"eth.type",
"icmp.checksum",
"icmpv6.checksum",
"igmp.checksum",
"igmp.type",
"ip.id",
"ipv6.flow",
"ipv6.tclass",
"ntp.flags",
"stun.att.crc32",
"stun.att.transp",
"stun.type",
"stun.type.class",
"stun.type.method",
"stun.type.method-assignment",
"tcp.checksum",
"tcp.flags",
"tls.handshake.server_curve_type",
"tls.handshake.server_named_curve",
"tls.handshake.version",
"udp.checksum",
"dhcp.hw.type",
"ipv6.opt.type","tls.handshake.sig_hash_alg",
"ip.dsfield",
"ip.flags",
"stun.att.family",
"stun.att.ipv4-xord",
"stun.att.port-xord",
"ipv6.opt.type.rest"]

# function that holds the first value and discards the second value for repeated values
def hex2dec(val):
    clean=[]
    for v in val:
        try:
            clean.append(int(v, 16))
        except:
            clean.append(v)  
    return clean


In [34]:
path="pcaps"
pcaps=find_the_way(path,".pcap")
pcaps

['pcaps\\16-09-29.pcap',
 'pcaps\\16-10-03.pcap',
 'pcaps\\16-11-18.pcap',
 'pcaps\\16-11-22.pcap',
 'pcaps\\18-06-11.pcap',
 'pcaps\\18-06-14.pcap',
 'pcaps\\18-10-13.pcap',
 'pcaps\\18-10-16.pcap']

In [35]:
#### pcap to csv using wireshark
#### Feature extraction main part

for numero, pp in enumerate(pcaps):
    print(f"{numero+1}/{len(pcaps)}   ---> {pp}   ")
    killthemall()
    name=pp.replace("\\","/")
    print("STEP - 1 - (PCAP2CSV)")
    for p,feature in enumerate (tqdm(features)):
        line=f"tshark -r {name} -E header=y -E separator=\"|\" -T fields -e "
        for i in feature:
            line=line+ i + " -e "
        line= line[:-3] + f"> ./temp/{p}.csv"
        os.system(line)
    
    files_add=find_the_way('./temp/',".csv")
    files_add
        
    csvname=name.replace("pcap","csv")
    folder(csvname[:csvname.rfind("/")])
    flag=1
    print("STEP - 2 - (CSV Merge)")
    for i in  tqdm(files_add):
        try:
            if flag:
                df=pd.read_csv(i,sep="|")
                #df=df.dropna(axis=1, how='all')
                df.to_csv(csvname,sep="|", index=False)
                flag=0
            else:
                df=pd.read_csv(csvname,sep="|")
                df1=pd.read_csv(i,sep="|")
                #df1=df1.dropna(axis=1, how='all')
    
                result = pd.concat([df, df1], axis=1)
                result.to_csv(csvname,sep="|", index=False)
                #result.to_csv(f"nihai{flag}.csv", index=False)
        except:print("error",i)




1/8   ---> pcaps\16-09-29.pcap   
STEP - 1 - (PCAP2CSV)


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:03<00:00,  2.07it/s]


STEP - 2 - (CSV Merge)


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:07<00:00,  1.13s/it]


2/8   ---> pcaps\16-10-03.pcap   
STEP - 1 - (PCAP2CSV)


...

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:08<00:00,  1.14s/it]


8/8   ---> pcaps\18-10-16.pcap   
STEP - 1 - (PCAP2CSV)


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:03<00:00,  2.06it/s]


STEP - 2 - (CSV Merge)


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:07<00:00,  1.09s/it]


In [36]:
# MAC address of devices and their names. It will use for labelling
macs={'d0:52:a8:00:67:5e': 'Smart Things',
 '44:65:0d:56:cc:d3': 'Amazon Echo',
 '70:ee:50:18:34:43': 'Netatmo Welcome',
 'f4:f2:6d:93:51:f1': 'TP-Link Day Night Cloud camera',
 '00:16:6c:ab:6b:88': 'Samsung SmartCam',
 '30:8c:fb:2f:e4:b2': 'Dropcam',
 'ec:1a:59:79:f4:89': 'Belkin Wemo switch',
 '50:c7:bf:00:56:39': 'TP-Link Smart plug',
 '74:c6:3b:29:d7:1d': 'iHome',
 'ec:1a:59:83:28:11': 'Belkin wemo motion sensor',
 '18:b4:30:25:be:e4': 'NEST Protect smoke alarm',
 '70:ee:50:03:b8:ac': 'Netatmo weather station',
 'd0:73:d5:01:83:08': 'Light Bulbs LiFX Smart Bulb',
 '18:b7:9e:02:20:44': 'Triby Speaker',
 'e0:76:d0:33:bb:85': 'PIX-STAR Photo-frame',
 '70:5a:0f:e4:9b:c0': 'HP Printer',
 '08:21:ef:3b:fc:e3': 'Samsung Galaxy Tab',
 'ac:bc:32:d4:6f:2f': 'MacBook',
 '14:cc:20:51:33:ea': 'TPLink Router Bridge LAN',
 '00:17:88:2b:9a:25': 'Phillip Hue Lightbulb',
# '7c:70:bc:5d:5e:dc': 'Canary Camera',
 '70:88:6b:10:0f:c6': 'Awair air quality monitor'}


In [37]:
#### General corrections in CSV files (payload entropy, port class, merging of IPv4 and IPv6 properties, Labelling, etc.)
#### nan values filled with -9999
#### inf values filled with max(uint32)
files_add=find_the_way('./csvs/',".csv")
for name in tqdm(files_add):
    df=pd.read_csv(name,sep="|")
    for d in dubs:
        temp=df[d].values
        temp=comma_cleaner(temp)
        df[d]=temp
    for h in hexvalues:
        temp=df[h].values
        temp=hex2dec(temp)
        df[h]=temp
    df['tcp.options']=df['tcp.options'].astype(np.float32)
    df['tcp.options']=df['tcp.options'].replace([np.inf, -np.inf], np.iinfo(np.uint32).max)  

    for i in str_cols:
        temp=i.replace(".","_")
        val=df[i]
        val=val.replace(strs[temp])
        temp=hex2dec(temp)
        df[i]=val
    # Sütunları birleştir
    df['Payload_Entropy'] = df["udp.payload"].combine_first(df["tcp.payload"])
    del df["udp.payload"]
    del df["tcp.payload"]
    temp=[]
    for i in (df['Payload_Entropy'].values):
        temp.append(shannon(i))
    df['Payload_Entropy']=temp


    df['dstport'] = df['udp.dstport'].combine_first(df['udp.dstport'])
    df['srcport'] = df['udp.srcport'].combine_first(df['udp.srcport'])
    
    temp=[]
    for i in (df['dstport'].values):
        temp.append(port_class(i))
    df['dstport_class']=temp
    
    temp=[]
    for i in (df['srcport'].values):
        temp.append(port_class(i))
    df['srcport_class']=temp


    df['dstport'] = df['udp.dstport'].combine_first(df['udp.dstport'])
    df['srcport'] = df['udp.srcport'].combine_first(df['udp.srcport'])
    #df['ip.dst'] = df['ip.dst'].combine_first(df['ipv6.dst'])
   # df['ip.src'] = df['ip.src'].combine_first(df['ipv6.src'])
    
    df['ip.bogus_ip_length'] = df['ip.bogus_ip_length'].combine_first(df['ipv6.hlim'])
    df['ip.len'] = df['ip.len'].combine_first(df['ipv6.hopopts.nxt'])
    df['ip.opt.len'] = df['ip.opt.len'].combine_first(df['ipv6.nxt'])
    df['ip.opt.ra'] = df['ip.opt.ra'].combine_first(df['ipv6.opt.router_alert'])
    df['ip.opt.type'] = df['ip.opt.type'].combine_first(df['ipv6.plen'])
    df['ip.opt.type.class'] = df['ip.opt.type.class'].combine_first(df['ipv6.tclass.dscp'])
    df['ip.opt.type.copy'] = df['ip.opt.type.copy'].combine_first(df['ipv6.tclass.ecn'])
    df['ip.opt.type.number'] = df['ip.opt.type.number'].combine_first(df['ipv6.version'])
    

    del df['ipv6.hlim']
    del df['ipv6.hopopts.nxt']
    del df['ipv6.nxt']
    del df['ipv6.opt.router_alert']
    del df['ipv6.plen']
    del df['ipv6.tclass.dscp']
    del df['ipv6.tclass.ecn']
    del df['ipv6.version']
    df=df.fillna(-9999)
    df["Label"]=df["eth.src"].replace(macs)
    df.to_csv(name,index=False)

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:36<00:00,  4.56s/it]


# Merge Small CSVs

### if you have split the pcap files into small sizes, you can use this section to merge their csv output

In [38]:
days=[]
them=find_the_way("./csvs/","small")
for i in them:
    days.append(i[15:23])
days=list(set(days))
days

[]

In [39]:
def csvmerger(output):
    ths = open(f"{output}.csv", "w")
    flag=1
    them=find_the_way("./csvs/",output)
    for t in them:
        print(t)
        with open(t, "r") as file:
            while True:
                line=file.readline()
                if line=="":break
                if not line.startswith("dns"):
                    #print(line)
                    ths.write(line)
                    #print(line)
                    #break
                else:
                    if flag:
                        ths.write(line)
                        flag=0
        os.remove(t)
    ths.close()

In [40]:
for i in days:
    csvmerger(i)

# Function that brings together disjoint csv files for ML evaluation. This function also reduces the csv file size by eliminating unnecessary or unusable samples (irrelevant device names)

In [41]:
devices=list(macs.values())
#devices

In [42]:
them=find_the_way("./csvs/",".csv")

In [43]:
proError=[]
for t in them:
    #main_labels="dns.time,http.time,icmp.data_time_relative,icmp.resptime,stun.time,tcp.ack,tcp.ack_raw,tcp.analysis.ack_rtt,tcp.analysis.initial_rtt,tcp.analysis.rto,tcp.nxtseq,tcp.options.timestamp.tsecr,tcp.options.timestamp.tsval,tcp.seq,tcp.seq_raw,tcp.time_delta,tcp.time_relative,udp.time_delta,udp.time_relative,arp.hw.size,arp.hw.type,arp.isannouncement,arp.isgratuitous,arp.isprobe,arp.opcode,arp.proto.size,arp.seconds-since-duplicate-address-frame,dhcp.flags.bc,dhcp.hops,dhcp.hw.addr_padding,dhcp.hw.len,dhcp.option.dhcp,dhcp.option.dhcp_max_message_size,dhcp.option.end,dhcp.option.interface_mtu,dhcp.option.ip_address_lease_time,dhcp.option.padding,dhcp.option.rebinding_time_value,dhcp.option.renewal_time_value,dhcp.secs,dhcp.type,dns.count.add_rr,dns.count.answers,dns.count.auth_rr,dns.count.queries,dns.extraneous,dns.extraneous.data,dns.extraneous.length,dns.flags.authenticated,dns.flags.authoritative,dns.flags.checkdisable,dns.flags.opcode,dns.flags.rcode,dns.flags.recavail,dns.flags.recdesired,dns.flags.response,dns.flags.truncated,dns.flags.z,dns.response_to,dns.retransmission,dns.retransmit_request,dns.retransmit_request_in,dns.retransmit_response,dns.retransmit_response_in,dns.soa.expire_limit,dns.soa.minimum_ttl,dns.soa.refresh_interval,dns.soa.retry_interval,dns.soa.serial_number,dns.unsolicited,eapol.keydes.key_iv,eapol.keydes.key_len,eapol.keydes.replay_counter,eapol.keydes.type,eapol.len,eapol.type,eapol.version,eth.dst.ig,eth.dst.lg,eth.dst.oui,eth.fcs.status,eth.len,eth.padding,eth.padding_bad,eth.src.ig,eth.src.lg,eth.src.oui,http.chat,http.content_length,http.content_length_header,http.notification,http.prev_request_in,http.prev_response_in,http.request,http.request_in,http.request_number,http.response,http.response.code,http.response_number,icmp.checksum.status,icmp.code,icmp.ident,icmp.ident_le,icmp.resp_to,icmp.seq,icmp.seq_le,icmp.type,icmp.unused,icmpv6.checksum.status,icmpv6.code,icmpv6.mldr.nb_mcast_records,icmpv6.opt,icmpv6.opt.length,icmpv6.opt.type,icmpv6.reserved,icmpv6.type,igmp.checksum.status,igmp.max_resp,igmp.num_grp_recs,igmp.version,ip.bogus_ip_length,ip.len,ip.opt.len,ip.opt.ra,ip.opt.type,ip.opt.type.class,ip.opt.type.copy,ip.opt.type.number,ntp.flags.li,ntp.flags.mode,ntp.flags.vn,ntp.ppoll,ntp.precision,ntp.rootdelay,ntp.rootdispersion,ntp.stratum,stun.att.crc32.status,stun.att.error,stun.att.error.class,stun.att.lifetime,stun.attributes,stun.length,stun.network_version,stun.response-to,tcp.analysis,tcp.analysis.ack_lost_segment,tcp.analysis.acks_frame,tcp.analysis.bytes_in_flight,tcp.analysis.duplicate_ack_frame,tcp.analysis.duplicate_ack_num,tcp.analysis.flags,tcp.analysis.keep_alive,tcp.analysis.keep_alive_ack,tcp.analysis.lost_segment,tcp.analysis.out_of_order,tcp.analysis.push_bytes_sent,tcp.analysis.retransmission,tcp.analysis.reused_ports,tcp.analysis.rto_frame,tcp.analysis.spurious_retransmission,tcp.analysis.window_update,tcp.checksum.status,tcp.completeness,tcp.connection.fin,tcp.connection.fin_active,tcp.connection.fin_passive,tcp.connection.rst,tcp.connection.syn,tcp.connection.synack,tcp.dstport,tcp.fin_retransmission,tcp.flags.ack,tcp.flags.ae,tcp.flags.cwr,tcp.flags.ece,tcp.flags.fin,tcp.flags.push,tcp.flags.res,tcp.flags.reset,tcp.flags.syn,tcp.flags.urg,tcp.hdr_len,tcp.len,tcp.options.mss_val,tcp.options.sack.count,tcp.options.sack.dsack,tcp.options.sack.dsack_le,tcp.options.sack.dsack_re,tcp.options.sack_le,tcp.options.sack_perm,tcp.options.sack_re,tcp.options.wscale.multiplier,tcp.options.wscale.shift,tcp.pdu.size,tcp.reassembled.length,tcp.segment.count,tcp.segment.overlap,tcp.segments,tcp.srcport,tcp.stream,tcp.urgent_pointer,tcp.window_size,tcp.window_size_scalefactor,tcp.window_size_value,tls.alert_message,tls.change_cipher_spec,tls.handshake.certificates,tls.handshake.certificates_length,tls.handshake.challenge,tls.handshake.challenge_length,tls.handshake.cipher_spec_len,tls.handshake.cipher_suites_length,tls.handshake.ciphersuites,tls.handshake.client_point_len,tls.handshake.comp_method,tls.handshake.comp_methods,tls.handshake.comp_methods_length,tls.handshake.epms_len,tls.handshake.extension.heartbeat.mode,tls.handshake.extensions_ec_point_formats,tls.handshake.extensions_ec_point_formats_length,tls.handshake.extensions_length,tls.handshake.extensions_reneg_info_len,tls.handshake.extensions_server_name_len,tls.handshake.extensions_server_name_list_len,tls.handshake.extensions_server_name_type,tls.handshake.extensions_supported_groups,tls.handshake.extensions_supported_groups_length,tls.handshake.g_len,tls.handshake.p_len,tls.handshake.server_point_len,tls.handshake.session_id_length,tls.handshake.session_ticket_length,tls.handshake.session_ticket_lifetime_hint,tls.handshake.sig_hash_alg_len,tls.handshake.sig_hash_algs,tls.handshake.sig_len,tls.handshake.yc_len,tls.handshake.ys_len,tls.resumed,tls.sct.scts_length,udp.checksum.status,udp.dstport,udp.length,udp.length.bad,udp.possible_traceroute,udp.srcport,udp.stream,eth.dst,eth.src,stun.cookie,stun.id,stun.value,tcp.options,tcp.options.mss,tcp.options.wscale,ntp.refid,arp.proto.type,dhcp.flags,dhcp.flags.reserved,dhcp.id,dns.flags,dns.id,eth.fcs,eth.type,icmp.checksum,icmpv6.checksum,igmp.checksum,igmp.type,ip.id,ipv6.flow,ipv6.tclass,ntp.flags,stun.att.crc32,stun.att.transp,stun.type,stun.type.class,stun.type.method,stun.type.method-assignment,tcp.checksum,tcp.flags,tls.handshake.server_curve_type,tls.handshake.server_named_curve,tls.handshake.version,udp.checksum,http.connection,http.request.method,http.response.code.desc,http.response.phrase,tcp.flags.str,_ws.col.Protocol,tls.handshake.sig_hash_alg,dhcp.hw.type,ip.dsfield,ip.flags,ipv6.opt.type,ipv6.opt.type.rest,stun.att.family,stun.att.ipv4-xord,stun.att.port-xord,tcp.segment,arp.duplicate-address-detected,arp.duplicate-address-frame,dns.srv.port,dns.srv.priority,dns.srv.weight,eth.addr.oui,eth.ig,eth.lg,ip.checksum.status,ip.dsfield.dscp,ip.dsfield.ecn,ip.flags.df,ip.flags.mf,ip.flags.rb,ip.frag_offset,ip.hdr_len,ip.proto,ip.ttl,ip.version,ipv6.opt,ipv6.opt.length,ipv6.opt.type.action,ipv6.opt.type.change,stun.att.padding,stun.att.port,stun.att.reserved,tcp.analysis.duplicate_ack,tls.record.length,Payload_Entropy,dstport,srcport,dstport_class,srcport_class,Label\n"
    ths = open(t.replace(".csv","_.csv"), "w")
    #ths.write(main_labels)

    with open(t, "r") as file:
        line=file.readline()
        ths.write(line)
        while True:
            try:
                line=file.readline()
                if line=="":break
                temp=line[:-1]
                temp=temp.split(",")
                if temp[-1] in devices:
                    if int(temp[290])!=128:
                        ths.write(line)
            except:
                if temp[290] not in proError:
                    print(temp[290])
                    proError.append(temp[290])

    ths.close()
    os.remove(t)
    

-9999.0


In [44]:
files_add=find_the_way("./csvs/",".csv")
for f in files_add:
    output=f.replace("_.csv",".csv")
    os.rename(f,output)

# In order to cope with missing devices in the experimental environment, appropriate sessions were combined to obtain sessions containing all devices.

In [45]:
them={"DI-S1": ['./csvs/16-09-29.csv', './csvs/16-11-18.csv'],
"DI-S2": ['./csvs/16-10-03.csv','./csvs/16-11-22.csv'],
"AD-S1": ['./csvs/18-06-14.csv',  './csvs/18-10-13.csv'],
"AD-S2": [ './csvs/18-06-11.csv','./csvs/18-10-16.csv']}

In [46]:
for output in them:
    ths = open(f"./csvs/{output}.csv", "w")
    flag=1

    for t in them[output]:
        with open(t, "r") as file:
            while True:
                line=file.readline()
                if line=="":break
                if not line.startswith("dns"):
                    #print(line)
                    ths.write(line)
                    #print(line)
                    #break
                else:
                    if flag:
                        ths.write(line)
                        flag=0
        os.remove(t)
    ths.close()

# CREATE SMALLER CSVs (1/20)

#### CSV files may be too large to be processed.  You can sample CSV files using these sections.
#### Since sampling uses proportions, the csvs reduced in this type of sampling will reflect real-life examples.

### USING PANDAS

In [68]:
files_add=find_the_way("./csvs/",".csv")
files_add

['./csvs/AD-S1.csv',
 './csvs/AD-S2.csv',
 './csvs/DI-S1.csv',
 './csvs/DI-S2.csv']

In [69]:
folder("small")

In [70]:
import random
random.seed(12)

In [71]:
for f in files_add:
    print(f)
    name=f.replace("./csvs","./small")
    ths = open(name, "w")
    with open(f, "r") as file:
        line=file.readline()
        ths.write(line)
        while True:
            line=file.readline()
            if line=="":break
            rnd=random.randint(0,20)
            if rnd==12:
                ths.write(line)
    ths.close()       

./csvs/AD-S1.csv
./csvs/AD-S2.csv
./csvs/DI-S1.csv
./csvs/DI-S2.csv


In [72]:
files_add=find_the_way("./small/",".csv")

In [73]:
for i in files_add:
    df=pd.read_csv(i,usecols=["Label"])
    temp=df.groupby("Label").size()
    print(len(temp),temp)
    print("#"*120,"\n\n\n")

21 Label
Amazon Echo                         2821
Awair air quality monitor            739
Belkin Wemo switch                  1500
Belkin wemo motion sensor          25267
Dropcam                            14935
HP Printer                           354
Light Bulbs LiFX Smart Bulb         1303
MacBook                               93
NEST Protect smoke alarm              13
Netatmo Welcome                     4218
Netatmo weather station              314
PIX-STAR Photo-frame                 215
Phillip Hue Lightbulb               4479
Samsung Galaxy Tab                 12705
Samsung SmartCam                   12253
Smart Things                        1961
TP-Link Day Night Cloud camera       387
TP-Link Smart plug                   165
TPLink Router Bridge LAN          175530
Triby Speaker                       1518
iHome                                418
dtype: int64
21 Label
Amazon Echo                         1349
Awair air quality monitor            736
Belkin Wemo switch        

# Alternative method to shrink CSV
# CREATE SMALLER CSVs (choose 10000 each)

#### Using pandas on large CSV files can sometimes cause problems. In the following example I have developed a system that can sample very large files quickly and without problems.

In [14]:
them=find_the_way("./csvs/",".csv")
them

['./csvs/AD-S1.csv',
 './csvs/AD-S2.csv',
 './csvs/DI-S1.csv',
 './csvs/DI-S2.csv']

In [9]:
sizes={}
for t in them:
    hist= {} 
    with open(t, "r") as file:
        while True:
            line=file.readline()
            if line=="":break
            temp=line[:-1]
            temp=temp.split(",")
            temp=temp[-1]
            hist[temp]= hist.get(temp,0) +1

    print(hist)
    sizes[t]=hist
    

{'Label': 1, 'TPLink Router Bridge LAN': 3686119, 'Dropcam': 315620, 'Samsung SmartCam': 256673, 'Netatmo Welcome': 89183, 'Belkin wemo motion sensor': 527710, 'Amazon Echo': 59980, 'Awair air quality monitor': 15125, 'Smart Things': 42351, 'Belkin Wemo switch': 31059, 'iHome': 9043, 'HP Printer': 7402, 'TP-Link Smart plug': 3350, 'PIX-STAR Photo-frame': 4156, 'MacBook': 2197, 'Samsung Galaxy Tab': 263823, 'Triby Speaker': 31746, 'Light Bulbs LiFX Smart Bulb': 27200, 'NEST Protect smoke alarm': 273, 'Phillip Hue Lightbulb': 96581, 'TP-Link Day Night Cloud camera': 8347, 'Netatmo weather station': 6791}
{'Label': 1, 'Dropcam': 267373, 'TPLink Router Bridge LAN': 3681953, 'Belkin wemo motion sensor': 514614, 'Samsung SmartCam': 253855, 'Smart Things': 42312, 'Netatmo Welcome': 36227, 'Awair air quality monitor': 15266, 'PIX-STAR Photo-frame': 4161, 'Belkin Wemo switch': 20733, 'HP Printer': 7299, 'iHome': 8901, 'TP-Link Smart plug': 3321, 'NEST Protect smoke alarm': 431, 'Phillip Hue Lig

In [10]:
sizes

{'./csvs/AD-S1.csv': {'Label': 1,
  'TPLink Router Bridge LAN': 3686119,
  'Dropcam': 315620,
  'Samsung SmartCam': 256673,
  'Netatmo Welcome': 89183,
  'Belkin wemo motion sensor': 527710,
  'Amazon Echo': 59980,
  'Awair air quality monitor': 15125,
  'Smart Things': 42351,
  'Belkin Wemo switch': 31059,
  'iHome': 9043,
  'HP Printer': 7402,
  'TP-Link Smart plug': 3350,
  'PIX-STAR Photo-frame': 4156,
  'MacBook': 2197,
  'Samsung Galaxy Tab': 263823,
  'Triby Speaker': 31746,
  'Light Bulbs LiFX Smart Bulb': 27200,
  'NEST Protect smoke alarm': 273,
  'Phillip Hue Lightbulb': 96581,
  'TP-Link Day Night Cloud camera': 8347,
  'Netatmo weather station': 6791},
 './csvs/AD-S2.csv': {'Label': 1,
  'Dropcam': 267373,
  'TPLink Router Bridge LAN': 3681953,
  'Belkin wemo motion sensor': 514614,
  'Samsung SmartCam': 253855,
  'Smart Things': 42312,
  'Netatmo Welcome': 36227,
  'Awair air quality monitor': 15266,
  'PIX-STAR Photo-frame': 4161,
  'Belkin Wemo switch': 20733,
  'HP Pri

In [20]:
limit=5000
for t in them:
    print(t)
    name=t.replace("./csvs","./small")
    ths = open(name, "w")
    with open(t, "r") as file:
        line=file.readline()
        ths.write(line)
        while True:
            line=file.readline()
            if line=="":break
            temp=line[:-1]
            temp=temp.split(",")
            temp=temp[-1]
            if sizes[t][temp]<limit:
                ths.write(line)
            else:
                temp=sizes[t][temp]
                temp=temp//limit
                rnd=random.randint(0,temp)
                if rnd==1:
                    ths.write(line)
                
    ths.close()       

./csvs/AD-S1.csv
./csvs/AD-S2.csv
./csvs/DI-S1.csv
./csvs/DI-S2.csv


In [21]:
files_add=find_the_way("./small/",".csv")

In [22]:
for i in files_add:
    df=pd.read_csv(i,usecols=["Label"])
    temp=df.groupby("Label").size()
    print(len(temp),temp)
    print("#"*120,"\n\n\n")

21 Label
Amazon Echo                       4569
Awair air quality monitor         3288
Belkin Wemo switch                3999
Belkin wemo motion sensor         4954
Dropcam                           4742
HP Printer                        2816
Light Bulbs LiFX Smart Bulb       4043
MacBook                           2189
NEST Protect smoke alarm           262
Netatmo Welcome                   4876
Netatmo weather station           2014
PIX-STAR Photo-frame              3533
Phillip Hue Lightbulb             4710
Samsung Galaxy Tab                4860
Samsung SmartCam                  4756
Smart Things                      3734
TP-Link Day Night Cloud camera    3401
TP-Link Smart plug                2437
TPLink Router Bridge LAN          4970
Triby Speaker                     4246
iHome                             3077
dtype: int64
######################################################################################################################## 



21 Label
Amazon Echo              

In [15]:
sizes={'./csvs/AD-S1.csv':{'Label': 1, 'TPLink Router Bridge LAN': 3686119, 'Dropcam': 315620, 'Samsung SmartCam': 256673, 'Netatmo Welcome': 89183, 'Belkin wemo motion sensor': 527710, 'Amazon Echo': 59980, 'Awair air quality monitor': 15125, 'Smart Things': 42351, 'Belkin Wemo switch': 31059, 'iHome': 9043, 'HP Printer': 7402, 'TP-Link Smart plug': 3350, 'PIX-STAR Photo-frame': 4156, 'MacBook': 2197, 'Samsung Galaxy Tab': 263823, 'Triby Speaker': 31746, 'Light Bulbs LiFX Smart Bulb': 27200, 'NEST Protect smoke alarm': 273, 'Phillip Hue Lightbulb': 96581, 'TP-Link Day Night Cloud camera': 8347, 'Netatmo weather station': 6791},
 './csvs/AD-S2.csv':{'Label': 1, 'Dropcam': 267373, 'TPLink Router Bridge LAN': 3681953, 'Belkin wemo motion sensor': 514614, 'Samsung SmartCam': 253855, 'Smart Things': 42312, 'Netatmo Welcome': 36227, 'Awair air quality monitor': 15266, 'PIX-STAR Photo-frame': 4161, 'Belkin Wemo switch': 20733, 'HP Printer': 7299, 'iHome': 8901, 'TP-Link Smart plug': 3321, 'NEST Protect smoke alarm': 431, 'Phillip Hue Lightbulb': 99306, 'TP-Link Day Night Cloud camera': 8580, 'Samsung Galaxy Tab': 257415, 'Amazon Echo': 28216, 'Triby Speaker': 6514, 'Light Bulbs LiFX Smart Bulb': 15195, 'Netatmo weather station': 6869, 'MacBook': 3020},
 './csvs/DI-S1.csv':{'Label': 1, 'Dropcam': 198230, 'TPLink Router Bridge LAN': 964339, 'Amazon Echo': 63314, 'Netatmo Welcome': 53493, 'Belkin Wemo switch': 66667, 'Triby Speaker': 10734, 'HP Printer': 14920, 'Light Bulbs LiFX Smart Bulb': 12446, 'Smart Things': 30208, 'iHome': 4592, 'Samsung SmartCam': 80942, 'Belkin wemo motion sensor': 101555, 'TP-Link Day Night Cloud camera': 8218, 'TP-Link Smart plug': 2665, 'PIX-STAR Photo-frame': 4447, 'Netatmo weather station': 13032, 'Samsung Galaxy Tab': 16457, 'NEST Protect smoke alarm': 219, 'MacBook': 34600, 'Awair air quality monitor': 20201, 'Phillip Hue Lightbulb': 39188},
 './csvs/DI-S2.csv':{'Label': 1, 'Dropcam': 244948, 'Samsung SmartCam': 90723, 'TPLink Router Bridge LAN': 1467386, 'Netatmo Welcome': 102418, 'Belkin wemo motion sensor': 109823, 'Smart Things': 29968, 'TP-Link Day Night Cloud camera': 6963, 'Amazon Echo': 64943, 'iHome': 7056, 'PIX-STAR Photo-frame': 2033, 'Triby Speaker': 10563, 'Belkin Wemo switch': 65635, 'Light Bulbs LiFX Smart Bulb': 21561, 'HP Printer': 19037, 'TP-Link Smart plug': 3478, 'Netatmo weather station': 14132, 'Samsung Galaxy Tab': 99943, 'NEST Protect smoke alarm': 363, 'Awair air quality monitor': 16256, 'Phillip Hue Lightbulb': 50022, 'MacBook': 72514}}


In [18]:
df=pd.read_csv(i)

In [19]:
for i,ii in enumerate(df.columns):
    print(i,ii)

0 dns.time
1 http.time
2 icmp.data_time_relative
3 icmp.resptime
4 stun.time
5 tcp.ack
6 tcp.ack_raw
7 tcp.analysis.ack_rtt
8 tcp.analysis.initial_rtt
9 tcp.analysis.rto
10 tcp.nxtseq
11 tcp.options.timestamp.tsecr
12 tcp.options.timestamp.tsval
13 tcp.seq
14 tcp.seq_raw
15 tcp.time_delta
16 tcp.time_relative
17 udp.time_delta
18 udp.time_relative
19 arp.hw.size
20 arp.hw.type
21 arp.isannouncement
22 arp.isgratuitous
23 arp.isprobe
24 arp.opcode
25 arp.proto.size
26 arp.seconds-since-duplicate-address-frame
27 dhcp.flags.bc
28 dhcp.hops
29 dhcp.hw.addr_padding
30 dhcp.hw.len
31 dhcp.option.dhcp
32 dhcp.option.dhcp_max_message_size
33 dhcp.option.end
34 dhcp.option.interface_mtu
35 dhcp.option.ip_address_lease_time
36 dhcp.option.padding
37 dhcp.option.rebinding_time_value
38 dhcp.option.renewal_time_value
39 dhcp.secs
40 dhcp.type
41 dns.count.add_rr
42 dns.count.answers
43 dns.count.auth_rr
44 dns.count.queries
45 dns.extraneous
46 dns.extraneous.data
47 dns.extraneous.length
48 dns.

In [20]:
df["_ws.col.Protocol"]

0           4
1          14
2           4
3         128
4          16
         ... 
161885     14
161886      9
161887     16
161888     12
161889    128
Name: _ws.col.Protocol, Length: 161890, dtype: object